# Deep Unmixing Autoencoder (DAEU)
Implementation of

B. Palsson, J. Sigurdsson, J. R. Sveinsson and M. O. Ulfarsson, "Hyperspectral Unmixing Using a Neural Network Autoencoder," in IEEE Access, vol. 6, pp. 25646-25656, 2018, doi: 10.1109/ACCESS.2018.2818280.

## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import initializers, constraints, layers, activations, regularizers
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import tensor_shape
from unmixing import HSI, plotEndmembers,SAD
from unmixing import plotEndmembersAndGT, plotAbundancesSimple, load_HSI, PlotWhileTraining
from sklearn.feature_extraction.image import extract_patches_2d
from scipy import io as sio
import os
import numpy as np
from numpy.linalg import inv
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
%matplotlib inline

## Use CPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Class SumToOne
Custom layer that enforces the ASC.

In [ ]:
class SumToOne(layers.Layer):
    def __init__(self, **kwargs):
        super(SumToOne, self).__init__(**kwargs)
        
    def call(self, x):
        x *= K.cast(x >= K.epsilon(), K.floatx())
        x = K.relu(x)
        x = x/(K.sum(x, axis=-1, keepdims=True)+K.epsilon())
        return x

## Layer with soft thresholding ReLU activation

In [ ]:
class SparseReLU(tf.keras.layers.Layer):
    def __init__(self,params):
        self.params=params
        super(SparseReLU, self).__init__()
        self.alpha = self.add_weight(shape=(self.params['num_endmembers'],),initializer=tf.keras.initializers.Zeros(),
        trainable=True, constraint=tf.keras.constraints.non_neg())
    def build(self, input_shape):
        self.alpha = self.add_weight(shape=input_shape[1:],initializer=tf.keras.initializers.Zeros(),
        trainable=True, constraint=tf.keras.constraints.non_neg())
        super(SparseReLU, self).build(input_shape)
    def call(self, x):
        return tf.keras.backend.relu(x - self.alpha)

## Class Autoencoder

In [ ]:
class Autoencoder(object):
    def __init__(self, params):
        self.data = None
        self.params = params
        self.is_deep = True
        self.model = self.create_model()
        self.model.compile(optimizer=self.params["optimizer"], loss=self.params["loss"])
        
        
    def create_model(self):
        use_bias = False
        n_end = self.params['num_endmembers']
        # Input layer
        Sparse_ReLU = SparseReLU(self.params)
        input_ = layers.Input(shape=(self.params['n_bands'],))
          
        if self.is_deep:
            encoded = layers.Dense(n_end * 9, use_bias=use_bias,
                            activation=self.params['activation'])(input_)
            encoded = layers.Dense(n_end * 6, use_bias=use_bias,
                            activation=self.params['activation'])(encoded)
            #encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dense(n_end * 3, use_bias=use_bias,activation=self.params['activation'])(encoded)
            #encoded = layers.BatchNormalization()(encoded)
            encoded = layers.Dense(n_end, use_bias=use_bias,
                            activation=self.params['activation'])(encoded)
        else:
            encoded = Dense(n_end, use_bias=use_bias, activation=self.params['activation'], activity_regularizer=None,
                            kernel_regularizer=None)(input_)
        # Utility Layers

        # Batch Normalization
        encoded = layers.BatchNormalization()(encoded)
        # Soft Thresholding
        encoded = Sparse_ReLU(encoded)
        # Sum To One (ASC)
        encoded = SumToOne(name='abundances')(encoded)

        # Gaussian Dropout
        decoded = layers.GaussianDropout(0.0045)(encoded)

        # Decoder
        decoded = layers.Dense(self.params['n_bands'], activation='linear', name='endmembers',
                        use_bias=False,
                        kernel_constraint=constraints.non_neg())(
            encoded)

        return tf.keras.Model(inputs=input_ , outputs=decoded)
    
    def fit(self,data,plot_every):
        plot_callback = PlotWhileTraining(plot_every,self.params['data'])
        return self.model.fit(
            x=data,
            y=data,
            batch_size=self.params["batch_size"],
            epochs=self.params["epochs"],
            callbacks=[plot_callback]
        )
        

    def get_endmembers(self):
        return self.model.layers[len(self.model.layers) - 1].get_weights()[0]

    def get_abundances(self):
        intermediate_layer_model = tf.keras.Model(
            inputs=self.model.input, outputs=self.model.get_layer("abundances").output
        )
        abundances = intermediate_layer_model.predict(self.params['data'].array())
        abundances = np.reshape(abundances,[self.params['data'].cols,self.params['data'].rows,self.params['num_endmembers']])
        
        return abundances

## Set Hyperparameters and load data

In [ ]:
#Dictonary of aliases for datasets. The first string is the key and second is value (name of matfile without .mat suffix)
#Useful when looping over datasets
datasetnames = {'Urban':'Urban4'} 


dataset = "Urban"

hsi = load_HSI(
    "./Datasets/" + datasetnames[dataset] + ".mat"
)


# Hyperparameters
num_endmembers = 4
num_spectra = 2000
batch_size = 6
learning_rate = 0.001
epochs = 40
loss = SAD
opt = tf.optimizers.RMSprop(learning_rate=learning_rate)

data = hsi.array()

# Hyperparameter dictionary
params = {
    "num_endmembers": num_endmembers,
    "batch_size": batch_size,
    "num_spectra": num_spectra,
    "data": hsi,
    "epochs": epochs,
    "n_bands": hsi.bands,
    "GT": hsi.gt,
    "lr": learning_rate,
    "optimizer": opt,
    "loss": loss,
    "activation":layers.LeakyReLU(0.1)
}

plot_every = 0 #Plot endmembers and abundance maps every x epochs. Set to 0 when running experiments. 

training_data = data[
    np.random.randint(0, data.shape[0], num_spectra), :
]


## Test Autoencoder

In [ ]:
autoencoder = Autoencoder(params)
autoencoder.fit(training_data,plot_every)
endmembers = autoencoder.get_endmembers()
abundances = autoencoder.get_abundances()
plotEndmembersAndGT(endmembers, hsi.gt)
plotAbundancesSimple(abundances,'abund.png')

## Run Experiment

In [ ]:
num_runs = 25
plot_every = 0
results_folder = './Results'
method_name = 'DAEU'
for dataset in ['Urban']:
    save_folder = results_folder+'/'+method_name+'/'+dataset
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    dataset_name = 'synthetic'

    hsi = load_HSI(
        "./Datasets/" + datasetnames[dataset] + ".mat"
    )
    data=hsi.array()
    params['data']=hsi
    params['n_bands']=hsi.bands

    for run in range(1,num_runs+1):
        training_data = data[np.random.randint(0, data.shape[0], num_spectra), :]
        params['opt']=tf.optimizers.RMSprop(learning_rate=learning_rate)
        save_name = dataset_name+'_run'+str(run)+'.mat'
        save_path = save_folder+'/'+save_name
        autoencoder = Autoencoder(params)
        autoencoder.fit(training_data,plot_every)
        endmembers = autoencoder.get_endmembers()
        abundances = autoencoder.get_abundances()
        plotEndmembersAndGT(endmembers, hsi.gt)
        plotAbundancesSimple(abundances,'abund.png')
        sio.savemat(save_path,{'M':endmembers,'A':abundances})
        del autoencoder